# Задание
В рамках этого задания мы будем создавать программу, которая получая на вход отзыв, будет предсказывать, является отзыв положительным или отрицательным. Делать мы будем это таким образом: мы возьмём некоторое число заранее размеченных как положительные или отрицательные отзывов, выделим те слова, которые встречаются только в положительных или только в отрицательных отзывах, и будем считать, каких слов в поступившем нам на проверку отзыве больше.

In [1]:
from pprint import pprint

In [96]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
import random
import re
import pandas as pd

In [48]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from collections import defaultdict
import nltk

In [3]:
from sklearn.metrics import accuracy_score

## Сбор данных
Сначала нам надо скачать дату -- соберите как минимум 60 (30 положительных и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления "тонального словаря" (чем больше отзывов, тем лучше) и 10 отзывов для проверки качества. 

- 2 балла в случае сбора путём парсинга
- 1 - если найдете уже готовые данные или просто закопипастите без парсинга

In [34]:
def comment_parser(com):
    pattern = '\<div class\=\"body\" itemprop\=\"text\"\>.+?\<\/div\>'
    find = re.search(pattern, str(com))
    if find != None:
        text = find[0]
        text = text.replace('<div class="body" itemprop="text">', '')
        text = text.replace('</div>', '')
    else:
        text = ''
    return text

In [30]:
# Создадим подключение и обкачаем страницы с отзывами

def crawler(cur_link, tone):
    ua = UserAgent(verify_ssl=False)
    headers = {'User-Agent': ua.random}
    session = requests.session()
    
    pages = []
    dataset = pd.DataFrame(columns=['text', 'tone'])

    response = session.get(cur_link, headers=headers)

    # достанем ссылку на каждое аниме
    # перейдём по ссылке и достанем каждый отзыв
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')

    # обходим все аниме на странице
    for f in soup.find_all('article'):
        new_anime = session.get(f['data-url'], headers=headers)
        text = comment_parser(new_anime.text)
        dataset = dataset.append({'text':text, 'tone':tone}, ignore_index=True)

    return dataset

In [41]:
positive_pages = ['https://shikimori.one/animes/43608-kaguya-sama-wa-kokurasetai-ultra-romantic/reviews/positive',
                  'https://shikimori.one/animes/z5114-fullmetal-alchemist-brotherhood/reviews/positive',
                  'https://shikimori.one/animes/z9253-steins-gate/reviews/positive',
                  'https://shikimori.one/animes/47778-kimetsu-no-yaiba-yuukaku-hen/reviews/positive',
                  'https://shikimori.one/animes/50265-spy-x-family/reviews/positive',
                  'https://shikimori.one/animes/15809-hataraku-maou-sama/reviews/positive',
                  'https://shikimori.one/animes/z2001-tengen-toppa-gurren-lagann/reviews/positive',
                  'https://shikimori.one/animes/z1535-death-note/reviews/positive',
                  'https://shikimori.one/animes/23283-zankyou-no-terror/reviews/positive',
                  'https://shikimori.one/animes/z13601-psycho-pass/reviews/positive'
                 ]
negative_pages = ['https://shikimori.one/animes/43608-kaguya-sama-wa-kokurasetai-ultra-romantic/reviews/negative', 
                  'https://shikimori.one/animes/z5114-fullmetal-alchemist-brotherhood/reviews/negative', 
                  'https://shikimori.one/animes/z9253-steins-gate/reviews/negative', 
                  'https://shikimori.one/animes/47778-kimetsu-no-yaiba-yuukaku-hen/reviews/negative', 
                  'https://shikimori.one/animes/50265-spy-x-family/reviews/negative',
                  'https://shikimori.one/animes/15809-hataraku-maou-sama/reviews/negative',
                  'https://shikimori.one/animes/z2001-tengen-toppa-gurren-lagann/reviews/negative',
                  'https://shikimori.one/animes/z1535-death-note/reviews/negative',
                  'https://shikimori.one/animes/23283-zankyou-no-terror/reviews/negative',
                  'https://shikimori.one/animes/z13601-psycho-pass/reviews/negative'
                 ]

In [42]:
data = pd.DataFrame(columns=['text', 'tone'])

for page in positive_pages:
    data = pd.concat([data, crawler(page, 0)], 
                     ignore_index=True)

for page in negative_pages:
    data = pd.concat([data, crawler(page, 1)], 
                     ignore_index=True)

In [43]:
# удалим пустые строки
new_data = data[data['text'] != '']
new_data

,text,tone
0,Ну вот мы и добрались до 3 сезона. В принципе ...,0
1,"Не знаю, что на меня нашло, и почему я пишу на...",0
2,"Сериал, начинавшийся в первом сезоне как прост...",0
3,"Всё та же <a href=""https://shikimori.one/anime...",0
4,10/10 о боже как же это хорошо...<br>Ну а вооб...,0
...,...,...
131,"Интересная идея, потрясающий город и хорошая а...",1
132,Очень классный киберпанк с налётом антиутопии....,1
133,Музыкальное сопровождение:5/10<br>Персонажи:3/...,1
134,Аниме с множественными дырами. Мир тотального ...,1


## Препроцессинг
Токенизируйте слова, приведите их к нижнему регистру и к начальной форме.

- 1 балл за токенизацию
- 1 балл за начальную форму

In [45]:
def tok_lem(text):
    # очистка от тегов
    text = re.sub('<.+?>', '', text)
    
    # токенизация и начальная форма
    lemmas = ''
    lem_list = []
    for word in nltk.word_tokenize(text.lower()):
        lem = morph.parse(word)[0].normal_form
        lemmas += lem + ' '
        lem_list.append(lem)
    return lemmas, lem_list

In [54]:
processed = []
lem_processed = []

lem_sets = defaultdict(int)
for t in data['text']:
    tok_lem_resp = tok_lem(t)
    processed.append(tok_lem_resp[0])
    lem_processed.append(tok_lem_resp[1])
    for el in tok_lem_resp[1]:
        lem_sets[el] += 1

data['tokenized'] = processed
data['lem_dict'] = lem_processed

## Изучение данных
Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку).
- 2 балла

In [57]:
print('positive: ' + str(len(data[data['tone']==0])))
print('negative: ' + str(len(data[data['tone']==1])))

positive: 80
negative: 56


In [70]:
def grouper(df, i):
    # достанем множество слов из как-то окрашеных отзывов
    category_words = defaultdict(int)
    
    # сколько забираем на обучение
    num = {0:40, 1:28}
    
    category = df.groupby('tone').get_group(i)['lem_dict'][:num[i]]
    for c in category:
        for word in c:
            category_words[word] += 1
    return category_words

# это словари {слово:абсолютная частотность}
# при этом используем только на части отзывов
# остальные - тестовая выборка
positive_words = grouper(data, 0) 
negative_words = grouper(data, 1) 

In [89]:
# конечные множества
only_positive = defaultdict(int)
only_negative = defaultdict(int)

in_both = set(positive_words.keys()) & set(negative_words.keys())

for word in positive_words.keys():
    if positive_words[word] > 3 and word not in in_both:
        only_positive[word] += positive_words[word]

for word in negative_words.keys():
    if negative_words[word] > 3 and word not in in_both:
        only_negative[word] += negative_words[word]

In [90]:
pprint(only_positive)

defaultdict(<class 'int'>,
            {'1-й': 5,
             '1.': 4,
             '2003': 7,
             'адаптация': 9,
             'алхимик': 38,
             'альтернативный': 6,
             'близкий': 4,
             'братство': 30,
             'великолепный': 5,
             'внешность': 5,
             'врата': 5,
             'вырасти': 4,
             'гинтама': 8,
             'девочка': 4,
             'достойный': 4,
             'ждать': 8,
             'завоеватель': 5,
             'идеал': 4,
             'иметься': 4,
             'каговать': 4,
             'кандидат': 8,
             'картина': 5,
             'класс': 6,
             'лаба': 4,
             'лично': 7,
             'материал': 4,
             'минус': 8,
             'мнение': 6,
             'многие': 7,
             'музыкальный': 5,
             'найти': 4,
             'нана': 6,
             'нацумэ': 5,
             'начинаться': 5,
             'неожиданный': 4,
             'озвучка': 

In [91]:
pprint(only_negative)

defaultdict(<class 'int'>,
            {'#': 6,
             '*': 11,
             'бездарный': 4,
             'буквально': 5,
             'видимо': 4,
             'выполнять': 4,
             'гз': 4,
             'добрый': 7,
             'женщина': 5,
             'земля': 4,
             'зеница': 5,
             'иносукэ': 5,
             'как-то': 5,
             'клоунада': 4,
             'логика': 4,
             'мужской': 4,
             'нету': 5,
             'огонь': 8,
             'попытаться': 4,
             'простить': 5,
             'прочитать': 4,
             'работать': 4,
             'равно': 4,
             'разве': 4,
             'рояль': 6,
             'сатана': 5,
             'сколько': 4,
             'смешной': 5,
             'способный': 4,
             'сражение': 4,
             'тупой': 4,
             'тьма': 5,
             'тэнген': 6,
             'убивать': 4,
             'фишка': 4,
             'хаширо': 8,
             'целый': 6,
   

## Определение тональности
Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy
- 1 балл - за коректно работающую функцию
- 1 балл - за подсчёт accuracy

In [97]:
def tone_detection(review, opw, onw):
    # за каждое слово будем прибавлять его абсолютную
    # частотность из opw / onw (словари слов только
    # из положительных / отрицательных отзывов)
    pos_score = 0
    neg_score = 0
    for word in review:
        pos_score += opw[word]
        neg_score += onw[word]
    
    if pos_score > neg_score:
        return 0
    elif neg_score > pos_score:
        return 1
    else:
        # иначе 50% accuracy
        return random.randint(0,1)

In [110]:
y_true = list(data['tone'][40:80])
y_true.extend(list(data['tone'][108:]))

y_pred = []
for r in data['lem_dict'][40:80]:
    y_pred.append(tone_detection(r, only_positive, only_negative))
for r in data['lem_dict'][108:]:
    y_pred.append(tone_detection(r, only_positive, only_negative))

accuracy_score(y_true, y_pred)

0.5588235294117647

## Рефлексия
Предложите как минимум 2 способа улучшить эту программу с помощью добавления к ней любых мулек 
- 1 балл за описание словами
- 2 балла - если реализуете хотя бы один способ

### Способы
1. Убрать цифры и символы из учёта.
2. Брать из каждого аниме одинаковое количество отзывов.
3. Взять одинаковое количество положительных и отрицательных отзывов.
4. Убрать именованные сущности.
5. Придумать адекватный алгоритм для случаев, когда слов не нашлось (например, составить словарь потенциально положительных и отрицательных флагов.

In [111]:
# попробуем убрать цифры и символы

only_positive = defaultdict(int)
only_negative = defaultdict(int)

in_both = set(positive_words.keys()) & set(negative_words.keys())

for word in positive_words.keys():
    if positive_words[word] > 3 and word not in in_both and word.isalpha():
        only_positive[word] += positive_words[word]

for word in negative_words.keys():
    if negative_words[word] > 3 and word not in in_both and word.isalpha():
        only_negative[word] += negative_words[word]

In [112]:
pprint(only_positive)

defaultdict(<class 'int'>,
            {'адаптация': 9,
             'алхимик': 38,
             'альтернативный': 6,
             'близкий': 4,
             'братство': 30,
             'великолепный': 5,
             'внешность': 5,
             'врата': 5,
             'вырасти': 4,
             'гинтама': 8,
             'девочка': 4,
             'достойный': 4,
             'ждать': 8,
             'завоеватель': 5,
             'идеал': 4,
             'иметься': 4,
             'каговать': 4,
             'кандидат': 8,
             'картина': 5,
             'класс': 6,
             'лаба': 4,
             'лично': 7,
             'материал': 4,
             'минус': 8,
             'мнение': 6,
             'многие': 7,
             'музыкальный': 5,
             'найти': 4,
             'нана': 6,
             'нацумэ': 5,
             'начинаться': 5,
             'неожиданный': 4,
             'озвучка': 5,
             'окаба': 5,
             'песня': 7,
             'пи

In [113]:
pprint(only_negative)

defaultdict(<class 'int'>,
            {'бездарный': 4,
             'буквально': 5,
             'видимо': 4,
             'выполнять': 4,
             'гз': 4,
             'добрый': 7,
             'женщина': 5,
             'земля': 4,
             'зеница': 5,
             'иносукэ': 5,
             'клоунада': 4,
             'логика': 4,
             'мужской': 4,
             'нету': 5,
             'огонь': 8,
             'попытаться': 4,
             'простить': 5,
             'прочитать': 4,
             'работать': 4,
             'равно': 4,
             'разве': 4,
             'рояль': 6,
             'сатана': 5,
             'сколько': 4,
             'смешной': 5,
             'способный': 4,
             'сражение': 4,
             'тупой': 4,
             'тьма': 5,
             'тэнген': 6,
             'убивать': 4,
             'фишка': 4,
             'хаширо': 8,
             'целый': 6,
             'чтоб': 4,
             'шпионский': 4,
             'яд': 

In [114]:
def tone_detection(review, opw, onw):
    # за каждое слово будем прибавлять его абсолютную
    # частотность из opw / onw (словари слов только
    # из положительных / отрицательных отзывов)
    pos_score = 0
    neg_score = 0
    for word in review:
        pos_score += opw[word]
        neg_score += onw[word]
    
    if pos_score > neg_score:
        return 0
    elif neg_score > pos_score:
        return 1
    else:
        # иначе 50% accuracy
        return random.randint(0,1)

In [115]:
y_true = list(data['tone'][40:80])
y_true.extend(list(data['tone'][108:]))

y_pred = []
for r in data['lem_dict'][40:80]:
    y_pred.append(tone_detection(r, only_positive, only_negative))
for r in data['lem_dict'][108:]:
    y_pred.append(tone_detection(r, only_positive, only_negative))

accuracy_score(y_true, y_pred)

0.6764705882352942

Ура, стало лучше! Правда, это могло произойти из-за более удачной работы последней функции (random.randit).